# HD 58647 (Herbig Ae): continuum PIONER+GRAVITY,  Br $\gamma$ emmission

- AMBER paper showing [Kurosawa et al. (2016)](https://ui.adsabs.harvard.edu/abs/2016MNRAS.457.2236K/abstract)
- PIONIER paper in the continuum [Lazareff et al. (2017)](https://ui.adsabs.harvard.edu/abs/2017A%26A...599A..85L/abstract)
- recent GRAVITY paper where Brackett $\gamma$ has been modeled [Bouarour et al. (2024)](https://arxiv.org/pdf/2312.08819.pdf)

In [2]:
# pip install ipympl
%matplotlib widget 
import numpy as np
import matplotlib.pyplot as plt
import os

import pmoired
print(pmoired.__version__)

1.2.4


# model PIONIER data downloaded from OIDb

- all Data with 6 spectral channels (2016, 2019)
- data with 3 spectral channels with smallest baselines

In [3]:
oi = pmoired.OI('./DATA/PIO*alibrated.fits')

loadOI: loading ./DATA/PIONI.2016-01-03T05 05 08.570_oidataCalibrated.fits
  > insname: "PIONIER_Pnat(1.5330000/1.7730000)" targname: "HD58647" pipeline: "pndrs_v3.53"
  > MJD: (1,) [ 57390.21035583429 .. 57390.21035583429 ]
  > D0-G2-J3-K0 | WL: (6,) [ 1.533 .. 1.773 ] um (R~34) {'T3': 4, 'VIS': 6, 'VIS2': 6} | TELL: False 
loadOI: loading ./DATA/PIONI.2016-01-03T05 29 43.476_oidataCalibrated.fits
  > insname: "PIONIER_Pnat(1.5330000/1.7730000)" targname: "HD58647" pipeline: "pndrs_v3.53"
  > MJD: (1,) [ 57390.22742455358 .. 57390.22742455358 ]
  > D0-G2-J3-K0 | WL: (6,) [ 1.533 .. 1.773 ] um (R~34) {'T3': 4, 'VIS': 6, 'VIS2': 6} | TELL: False 
loadOI: loading ./DATA/PIONI.2016-02-17T03 39 37.997_oidataCalibrated.fits
  > insname: "PIONIER_Pnat(1.5181904/1.7625174)" targname: "HD58647" pipeline: "pndrs_v3.53"
  > MJD: (1,) [ 57435.15096038368 .. 57435.15096038368 ]
  > D0-G2-J3-K0 | WL: (6,) [ 1.518 .. 1.763 ] um (R~34) {'T3': 4, 'VIS': 6, 'VIS2': 6} | TELL: False 
loadOI: loading ./D

## Make a star+rim+disk model "à la Lazareff"

See [PMOIRED tutorial](https://github.com/amerand/PMOIRED_examples) how to [build such model step by step](https://htmlpreview.github.io/?https://github.com/amerand/PMOIRED_examples/blob/main/html/EX2%20chromatic%20YSO%20disk.html)

In [ ]:
m = {'disk,F0':         0.0313, # +/- 0.0040
    'disk,PRO':        0.010, # +/- 0.036
    'disk,SPX':        2.09, # +/- 1.21
    'disk,az amp1':    0.57, # +/- 0.20
    'disk,az projang1':494.5, # +/- 12.0
    'disk,diamout':    30.03, # +/- 1.44
    'rim,F0':          0.297, # +/- 0.051
    'rim,SPX':         1.71, # +/- 0.55
    'rim,az amp1':     0.431, # +/- 0.045
    'rim,az amp2':     0.166, # +/- 0.085
    'rim,az projang1': -270.03, # +/- 1.52
    'rim,az projang2': -244.85, # +/- 4.34
    'rim,diamin':      1.89, # +/- 0.79
    'rim,diamout':     8.93, # +/- 1.00
    'rim,incl':        64.89, # +/- 0.89
    'rim,projang':     17.31, # +/- 1.41
    'disk,diamin':     '$rim,diamout',
    'disk,incl':       '$rim,incl',
    'disk,profile':    '$R**$disk,PRO',
    'disk,projang':    '$rim,projang',
    'disk,spectrum':   '$disk,F0*($WL/1.6)**$disk,SPX',
    'rim,profile':     'doughnut',
    'rim,spectrum':    '$rim,F0*($WL/1.6)**$rim,SPX',
    'star,spectrum':   '(1-$rim,F0-$disk,F0)*($WL/1.6)**-3.8',
    'star,ud':         0,
    }

# -- for the flux of the outer disk to decrease radially
prior = [('disk,PRO', '<=', 0)]

oi.setupFit({'obs':['T3PHI', 'V2'], 
              'max error':{'T3PHI':10} # remove a point with very large uncertainties
             })
# -- we do not fit the star as it is unresolved
oi.doFit(m, doNotFit=['star,ud'], prior=prior)
oi.show(imFov=14, imMax='99', imPow=0.5, imPlx=3.2831, logS=True)

[dpfit] 16 FITTED parameters: ['disk,F0', 'disk,PRO', 'disk,SPX', 'disk,az amp1', 'disk,az projang1', 'disk,diamout', 'rim,F0', 'rim,SPX', 'rim,az amp1', 'rim,az amp2', 'rim,az projang1', 'rim,az projang2', 'rim,diamin', 'rim,diamout', 'rim,incl', 'rim,projang']
[dpfit] epsfcn= 1e-08 ftol= 1e-05
[dpfit] using scipy.optimize.leastsq
[dpfit] Mon Jun 10 16:43:12 2024 001/000 CHI2: 5.9520e+01|
[dpfit] Mon Jun 10 16:43:22 2024 075/004 CHI2: 1.7332e+00|
[dpfit] Mon Jun 10 16:43:32 2024 145/009 CHI2: 1.1292e+00|
[dpfit] Mon Jun 10 16:43:42 2024 210/013 CHI2: 1.1066e+00|


## Add GRAVITY Data and fit continuum
Here the data are binned because we are just interested at the continuum. Data are public (but not the reduced one)

In [ ]:
oi.addData('./DATA/GRA*viscalibrated.fits', insname="GRAVITY_SC", binning=100)
# -- setup fit for GRAVITY only
oi.setupFit({'obs':['|V|', 'T3PHI']}, insname='GRAVITY_SC')
# -- show data with previoud model(fitted only to PIONIER data) 
oi.show(imFov=14, imMax='99', imPow=0.5, imWl0=[1.65, 2.2], imPlx=3.2831, logS=True)

In [ ]:
# -- fit all continuum data (PIONIER+GRAVITY)
oi.doFit(doNotFit=['star,ud'], prior=prior)
oi.show(imFov=14, imMax='99', imPow=0.5, imWl0=[1.65, 2.2], imPlx=3.2831, logS=True)

# Br $\gamma$ emmission in GRAVITY data

## First remove tellurics so we can use the spectrum as well

Interferometric quantities are not affected by the tellurics.

This takes quite some time, even if it is parallelised!

In [ ]:
from multiprocessing import Pool
from pmoired import tellcorr
files = sorted([os.path.join('./DATA', f) for f in filter(lambda f: f.startswith('G') and f.endswith('fits'), os.listdir('./DATA'))])

#with Pool() as p:
#    p.map(tellcorr.removeTellurics, files)

with Pool() as p:
    p.map(tellcorr.gravity, files)
    
tellcorr.showTellurics(files[0])

## re-load GRAVITY data only, in full spectral resolution, with correction from tellurics

In [ ]:
oi = pmoired.OI('./DATA/G*viscalibrated.fits', insname="GRAVITY_SC")
oi.fig = 100 # offset figure counter so we keep previous figures in the notebook

In [ ]:
# -- check data around Br gamma
oi.setupFit({'obs':['T3PHI', 'DPHI', 'NFLUX', '|V|'],
             'wl ranges':[(2.1661-0.005, 2.1661+0.005)],
            })
oi.show()
plt.figure(oi.fig-1).set_figheight(12)

## Add to the continnum model 2 blobs in emission around Brackett $\gamma$ 

In [ ]:
# -- continuum model from above
m = {'disk,F0':         0.0203, # +/- 0.0022
    'disk,PRO':        0.311, # +/- 0.070
    'disk,SPX':        3.07, # +/- 0.40
    'disk,az amp1':    0.36, # +/- 0.15
    'disk,az projang1':463.7, # +/- 23.4
    'disk,diamout':    40.21, # +/- 0.93
    'rim,F0':          0.2693, # +/- 0.0085
    'rim,SPX':         1.06, # +/- 0.17
    'rim,az amp1':     0.341, # +/- 0.030
    'rim,az amp2':     0.241, # +/- 0.061
    'rim,az projang1': -269.95, # +/- 1.88
    'rim,az projang2': -246.50, # +/- 5.29
    'rim,diamin':      0.00, # +/- 0.98
    'rim,diamout':     7.53, # +/- 0.14
    'rim,incl':        66.90, # +/- 0.94
    'rim,projang':     17.38, # +/- 0.71
    'disk,diamin':     '$rim,diamout',
    'disk,incl':       '$rim,incl',
    'disk,profile':    '$R**$disk,PRO',
    'disk,projang':    '$rim,projang',
    'disk,spectrum':   '$disk,F0*($WL/1.6)**$disk,SPX',
    'rim,profile':     'doughnut',
    'rim,spectrum':    '$rim,F0*($WL/1.6)**$rim,SPX',
    'star,spectrum':   '(1-$rim,F0-$disk,F0)*($WL/1.6)**-3.8',
    'star,ud':         0,
    }

oi.setupFit({'obs':['N|V|', 
                    #'T3PHI', 
                    'DPHI', 
                    'NFLUX'
                   ],
             'min relative error':{'NFLUX':0.02, 'N|V|':0.01},
             'min error':{'DPHI':1, 'T3PHI':2},
             'wl ranges':[(2.1661-0.005, 2.1661+0.005)],
            })

m.update({# -- blob1 size in mas
        'blob1,fwhm': 1,
        # -- continuum flux
        'blob1,f': 0.0, 
        # -- central wavelength, in um
        'blob1,line_0_wl0': '2.16+$blob1,DWL0/1000',
        'blob1,DWL0': 5.6,
        # -- gaussian line, FWHM in nm (not um!)
        'blob1,line_0_gaussian': 0.7,
        # -- line amplitude
        'blob1,line_0_f': 0.2,
        # -- blob position in mas
        'blob1,x': 0,
        'blob1,y': -1,
        # -- same for blob 2:
        'blob2,fwhm': 1,
        'blob2,f': 0.0,
        'blob2,line_0_wl0': '2.16+$blob2,DWL0/1000',
        'blob2,DWL0': 6.9,         
        'blob2,line_0_gaussian': 0.7,
        'blob2,line_0_f': 0.2,
        'blob2,x': 0,
        'blob2,y': 1,
         })

onlyShow = True

if onlyShow:
    # -- show model 
    oi.show(m, imFov=14, imMax='99', imPow=0.5, imWl0=[2.16, 2.1655, 2.16685], imPlx=3.2831)
    plt.figure(oi.fig-2).set_figheight(12)
else:
    # -- only fit blobs' parameters
    fitOnly = list(filter(lambda k: k.startswith('blob') and type(m[k])!=str and k.split(',')[1]!='f', m.keys()))
    # -- limit width of lines
    prior = [('blob1,fwhm', '>', 0.2),   
             ('blob1,fwhm', '<', 1.5),
             ('blob2,fwhm', '>', 0.2),
             ('blob2,fwhm', '<', 1.5),        
            ]    
    oi.doFit(m, fitOnly=fitOnly, prior=prior)
    oi.show(imFov=10, imMax='99', imPow=0.5, imWl0=[2.16, 2.1655, 2.16685], imPlx=3.2831)
    plt.figure(oi.fig-2).set_figheight(12)


## Add Keplerian disk to the continuum model

For a full intro to the Keplerian disk, check out [PMOIRED example](https://github.com/amerand/PMOIRED_examples) [#4](https://htmlpreview.github.io/?https://github.com/amerand/PMOIRED_examples/blob/main/html/EX4%20Be%20model%20comparison%20with%20AMHRA.html). This model is a re-implementation of the model described in [Delaa et al. (2011)](https://ui.adsabs.harvard.edu/abs/2011A%26A...529A..87D/abstract) and [Meilland et al. (2012)](https://ui.adsabs.harvard.edu/abs/2012A%26A...538A.110M/abstract), originaly developped to interpret VEGA and AMBER observations of Be stars.

In [ ]:
# -- continuum model from above
m = {'disk,F0':         0.0203, # +/- 0.0022
    'disk,PRO':        0.311, # +/- 0.070
    'disk,SPX':        3.07, # +/- 0.40
    'disk,az amp1':    0.36, # +/- 0.15
    'disk,az projang1':463.7, # +/- 23.4
    'disk,diamout':    40.21, # +/- 0.93
    'rim,F0':          0.2693, # +/- 0.0085
    'rim,SPX':         1.06, # +/- 0.17
    'rim,az amp1':     0.341, # +/- 0.030
    'rim,az amp2':     0.241, # +/- 0.061
    'rim,az projang1': -269.95, # +/- 1.88
    'rim,az projang2': -246.50, # +/- 5.29
    'rim,diamin':      0.00, # +/- 0.98
    'rim,diamout':     7.53, # +/- 0.14
    'rim,incl':        66.90, # +/- 0.94
    'rim,projang':     17.38, # +/- 0.71
    'disk,diamin':     '$rim,diamout',
    'disk,incl':       '$rim,incl',
    'disk,profile':    '$R**$disk,PRO',
    'disk,projang':    '$rim,projang',
    'disk,spectrum':   '$disk,F0*($WL/1.6)**$disk,SPX',
    'rim,profile':     'doughnut',
    'rim,spectrum':    '$rim,F0*($WL/1.6)**$rim,SPX',
    'star,spectrum':   '(1-$rim,F0-$disk,F0)*($WL/1.6)**-3.8',
    'star,ud':         0,
    }

m.update(
     {'kep,incl': m['rim,incl'], # only initialised to rim value, free fitted afterwards
     'kep,projang': m['rim,projang'], # only initialised to rim value, free fitted afterwards
     'kep,diamin': 1., # in mas 
     'kep,diamout': '0.5*($rim,diamout+$rim,diamin)', # in mas -> force emission to be in the cavity 
     'kep,Vin': 200, # velocity in km/s at diamin, sign defines the rotation direction
     'kep,line_0_rpow': -2, # power law for the radial intensity profile (in the line)
     'kep,line_0_EW': 1, # in nm, (assuming continnum flux is 1)
     'kep,beta': -0.5, # velocity power law: Keplerian is -0.5
     'kep,line_0_wl0': '2.16+$kep,DWL0/1000',
     'kep,DWL0':6.0,
     'kep,x':0.01, # better to initalise any value !=0
     'kep,y':0.01, # better to initalise any value !=0
     })

oi.setupFit({'obs':['N|V|', # normalised visibility
                    #'T3PHI', # very noisy and redundant with DPHI
                    'DPHI', 
                    'NFLUX'
                   ],
             'min relative error':{'NFLUX':0.02, 'N|V|':0.01},
             'min error':{'DPHI':1, 'T3PHI':2},
             'wl ranges':[(2.1661-0.005, 2.1661+0.005)],
            })

fitOnly = list(filter(lambda x: x.startswith('kep,')  and not 'beta' in x, m.keys()))

onlyShow=True
if onlyShow:
    oi.show(m, imFov=7, imMax='99', imPow=0.5, imWl0=[2.16, 2.16557, 2.166, 2.1669], imPlx=3.2831)
else:
    oi.doFit(m, fitOnly=fitOnly)
    oi.show(imFov=8, imMax='99', imPow=0.5, imWl0=[2.16, 2.16557, 2.166, 2.1669], imPlx=3.2831)

plt.figure(oi.fig-2).set_figheight(12)

In [ ]:
# -- stellar mass based on Keplerian disk velocity
plx = 3.2831 # in mas
a_au = (oi.bestfit['best']['kep,diamin']/2)/plx
P_yr = 2*np.pi*a_au*150e6/np.abs(oi.bestfit['best']['kep,Vin'])/3600/24/365.25
M_msun = a_au**3/P_yr**2
print("approx star's mass: %.1fMsun"%M_msun)